<a href="https://colab.research.google.com/github/nurcann23/AygazCompVisionBootcamp/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

# Enn güncel hali
import kagglehub
rrebirrth_animals_with_attributes_2_path = kagglehub.dataset_download('rrebirrth/animals-with-attributes-2')

print('Data source import complete.')

Resuming download from 969932800 bytes (12956809619 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/rrebirrth/animals-with-attributes-2?dataset_version_number=1 (969932800/13926742419) bytes left.


100%|██████████| 13.0G/13.0G [01:26<00:00, 149MB/s]

Extracting files...


Data source import complete.


In [30]:
import os

# Initialize the dictionary to hold paths for each animal
image_paths = {}

# Base directory for your dataset
base_path = "/root/.cache/kagglehub/datasets/rrebirrth/animals-with-attributes-2/versions/1/Animals_with_Attributes2/JPEGImages/"

# List of animals to search for
animals = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Traverse the directory structure
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        # Check if the current directory contains the animal's name
        if animal in dirname:
            # Initialize the list if the animal is encountered for the first time
            if animal not in image_paths:
                image_paths[animal] = []
            # Add all image paths for the current animal
            for filename in filenames:
                image_paths[animal].append(os.path.join(dirname, filename))

# Print the paths for verification
for animal, paths in image_paths.items():
    print(f"{animal}: {len(paths)} images")



giant+panda: 874 images
polar+bear: 868 images
rabbit: 1088 images
elephant: 1038 images
collie: 1028 images
squirrel: 1200 images
sheep: 1420 images
fox: 664 images
moose: 704 images
dolphin: 946 images


In [31]:
import tensorflow as tf

image_size = (128, 128)  # İstenen boyut
normalized_data = []

with tf.device('/GPU:0'):
    for cls, paths in image_paths.items():
        for img_path in paths:
            img_raw = tf.io.read_file(img_path)  # Resmi yükle
            img_tensor = tf.image.decode_jpeg(img_raw, channels=3)  # Resmi çöz
            img_resized = tf.image.resize(img_tensor, image_size)  # Yeniden boyutlandır
            img_normalized = img_resized / 255.0  # Normalize et
            normalized_data.append((img_normalized, cls))

print("Tüm resimler başarıyla normalize edildi.")


Tüm resimler başarıyla normalize edildi.


In [32]:
# İlk 5 normalleştirilmiş resmi ve etiketlerini göster
for i, (img, label) in enumerate(normalized_data[:5]):
    print(f"Resim {i + 1} - Etiket: {label}, Boyut: {img.shape}")


Resim 1 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 2 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 3 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 4 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 5 - Etiket: giant+panda, Boyut: (128, 128, 3)


In [33]:
from sklearn.model_selection import train_test_split
# Resimleri ve etiketleri ayır
X = [x[0] for x in normalized_data]
y = [x[1] for x in normalized_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("Eğitim ve test setleri başarıyla oluşturuldu.")
print(f"Eğitim setindeki resim sayısı: {len(X_train)}")
print(f"Test setindeki resim sayısı: {len(X_test)}")
print(f"Eğitim setindeki etiket sayısı: {len(y_train)}")
print(f"Test setindeki etiket sayısı: {len(y_test)}")

Eğitim ve test setleri başarıyla oluşturuldu.
Eğitim setindeki resim sayısı: 6881
Test setindeki resim sayısı: 2949
Eğitim setindeki etiket sayısı: 6881
Test setindeki etiket sayısı: 2949


In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Modeli oluştur
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # Daha küçük filtre
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),  # Daha küçük filtre
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),  # Daha az nöron
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Modeli derle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,393,610 (28.20 MB)

 Trainable params: 7,393,610 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

None


In [37]:
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

# Etiketleri one-hot encode
# Assuming 'animals' is your list of all classes
class_mapping = {cls: i for i, cls in enumerate(animals)}
y_train_enc = to_categorical([class_mapping[label] for label in y_train], num_classes=10)
y_test_enc = to_categorical([class_mapping[label] for label in y_test], num_classes=10)
# Modeli eğit

#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(np.array(X_train), np.array(y_train_enc), epochs=10, validation_data=(np.array(X_test), np.array(y_test_enc)))


Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.2302 - loss: 2.1509 - val_accuracy: 0.4405 - val_loss: 1.6611
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.4196 - loss: 1.6704 - val_accuracy: 0.5459 - val_loss: 1.3472
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5505 - loss: 1.3029 - val_accuracy: 0.5795 - val_loss: 1.2431
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.6344 - loss: 1.0656 - val_accuracy: 0.6043 - val_loss: 1.1943
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.6933 - loss: 0.9045 - val_accuracy: 0.6219 - val_loss: 1.1814
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.7337 - loss: 0.7841 - val_accuracy: 0.6246 - val_loss: 1.2035
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.7539 - loss: 0.7313 - val_accuracy: 0.6317 - val_loss: 1.2394
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.8350 - loss: 0.4537 - val_acc

In [38]:
from tensorflow.keras.applications import VGG16

# VGG16 modelini yükleyin, önceden eğitilmiş ağırlıklarla
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [39]:
# Tüm katmanları dondur
for layer in base_model.layers:
    layer.trainable = False


In [40]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Yeni katmanlar ekleyin
x = Flatten()(base_model.output)  # Özellik haritalarını düzleştir
x = Dense(128, activation='relu')(x)  # Yeni bir tam bağlantılı katman
x = Dropout(0.5)(x)  # Düzenlileştirme için Dropout
output = Dense(10, activation='softmax')(x)  # 10 sınıf için çıkış katmanı

# Yeni modeli tanımlayın
model = Model(inputs=base_model.input, outputs=output)


In [41]:
from tensorflow.keras.optimizers import Adam

# Modeli derleyin
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [42]:
# Modeli eğitin
model.fit(np.array(X_train), np.array(y_train_enc),
          epochs=10,
          validation_data=(np.array(X_test), np.array(y_test_enc)))


Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 31s 95ms/step - accuracy: 0.2777 - loss: 2.0827 - val_accuracy: 0.6863 - val_loss: 1.1466
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 14s 67ms/step - accuracy: 0.6164 - loss: 1.1973 - val_accuracy: 0.7399 - val_loss: 0.8835
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.6771 - loss: 0.9708 - val_accuracy: 0.7714 - val_loss: 0.7766
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.7249 - loss: 0.8439 - val_accuracy: 0.7959 - val_loss: 0.7073
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 14s 67ms/step - accuracy: 0.7771 - loss: 0.7251 - val_accuracy: 0.8009 - val_loss: 0.6684
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.7827 - loss: 0.6833 - val_accuracy: 0.8135 - val_loss: 0.6176
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 20s 60ms/step - accuracy: 0.7922 - loss: 0.6573 - val_accuracy: 0.8169 - val_loss: 0.6052
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 21s 60ms/step - accuracy: 0.8280 - loss: 0.5433 - 

In [43]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Öğrenme oranını düşürün ve modeli tekrar derleyin
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tuning yaparak yeniden eğitin
model.fit(np.array(X_train), np.array(y_train_enc),
          epochs=5,
          validation_data=(np.array(X_test), np.array(y_test_enc)))

Epoch 1/5
216/216 ━━━━━━━━━━━━━━━━━━━━ 28s 100ms/step - accuracy: 0.8717 - loss: 0.3916 - val_accuracy: 0.8545 - val_loss: 0.4541
Epoch 2/5
216/216 ━━━━━━━━━━━━━━━━━━━━ 18s 83ms/step - accuracy: 0.9045 - loss: 0.2844 - val_accuracy: 0.8661 - val_loss: 0.4193
Epoch 3/5
216/216 ━━━━━━━━━━━━━━━━━━━━ 19s 87ms/step - accuracy: 0.9174 - loss: 0.2488 - val_accuracy: 0.8728 - val_loss: 0.3943
Epoch 4/5
216/216 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.9486 - loss: 0.1768 - val_accuracy: 0.8749 - val_loss: 0.4006
Epoch 5/5
216/216 ━━━━━━━━━━━━━━━━━━━━ 21s 82ms/step - accuracy: 0.9534 - loss: 0.1500 - val_accuracy: 0.8840 - val_loss: 0.3802
